In [1]:
import torch
import torchvision
import numpy as np
import random

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
device = "cuda"

random_seed = 6
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./MNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./MNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 10)
        self.activation = nn.ELU()

        self.cf = nn.Linear(10, 50)

    def forward(self, x, pred_prob=None):
        x = self.activation(F.max_pool2d(self.conv1(x), 2))
        x = self.activation(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = self.activation(self.fc1(x))

        if pred_prob is not None:
            pred_prob = torch.nn.functional.softmax(pred_prob, 1)
            x += self.activation(self.cf(pred_prob))

        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [5]:
network = Net().to(device)
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

In [6]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [7]:
def inference_train(x, N = 3):
    preds = []
    pred = None
    for _ in range(N):
        pred = network(x, pred)
        preds.append(pred)
    return torch.cat(preds)

def inference_test(x, N = 3):
    preds = []
    pred = None
    for _ in range(N):
        pred = torch.nn.functional.softmax(network(x, pred), 1)
        preds.append(pred)
    return sum(preds) / N

In [8]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):

    data = data.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    output = inference_train(data)

    loss = F.nll_loss(output, target.tile(3))
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
#       torch.save(network.state_dict(), '/vanilla_model.pth')
#       torch.save(optimizer.state_dict(), '/vanilla_optimizer.pth')

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)
      output = inference_test(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [9]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

<ipython-input-4-bcdc46fa1a32>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: -0.0985, Accuracy: 984/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334754
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.720626
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.457343
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.351678
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.275641
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.132344
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.344049
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.164020
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.123514
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.147569
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.355349
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.158286
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.412770
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.272593
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.112904
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.102868
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.409741
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.048710
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.290640
Train Epoch: 1 [12160

In [17]:
for param_group in optimizer.param_groups:
    param_group['lr'] = 1e-3

for _ in range(2):
  train(epoch)
  test()

<ipython-input-4-bcdc46fa1a32>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000492
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.010211
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.000408
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.009567
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.001071
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.031634
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.039150
Train Epoch: 3 [4480/60000 (7%)]	Loss: 0.076876
Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.000230
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.001069
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.036519
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.013639
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.033857
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.183817
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.014725
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.005941
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.001451
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.006575
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.000261
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.144304
Train Epoch: 3 [12800/60000 (

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: -0.9867, Accuracy: 9891/10000 (99%)



In [18]:
train_fashion_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./FMNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_fashion_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./FMNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [19]:
# Computing OOD metrics

network.eval()

uncertainties = np.array([])
labels = np.array([])
eps = 1e-10

with torch.no_grad():
  for data, target in test_loader:

    data = data.to(device)
    prob = inference_test(data)
    uncertainty = (-prob * torch.log(prob + eps)).sum(dim=1).cpu().detach().numpy()
    label = np.zeros_like(uncertainty)

    uncertainties = np.concatenate([uncertainties, uncertainty])
    labels = np.concatenate([labels, label])

with torch.no_grad():
  for data, target in test_fashion_loader:

    data = data.to(device)

    prob = inference_test(data)
    uncertainty = (-prob * torch.log(prob + eps)).sum(dim=1).cpu().detach().numpy()
    label = np.ones_like(uncertainty)

    uncertainties = np.concatenate([uncertainties, uncertainty])
    labels = np.concatenate([labels, label])

<ipython-input-4-bcdc46fa1a32>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [20]:
import sklearn.metrics
roc_auc = sklearn.metrics.roc_auc_score(labels, uncertainties)
precision, recall, thresholds = sklearn.metrics.precision_recall_curve(labels, uncertainties)
pr_auc = sklearn.metrics.auc(recall, precision)
roc_auc, pr_auc

(0.974304505, 0.9800322945563024)